# Migration from the csv database to sql of the datasets

We are going to load the csv database, and load its information into the sql database automatically.

## Imports

In [30]:
import psycopg2
import numpy as np
#import a folder in the parent directory
import sys
sys.path.append('../')
import queries.queries as qrs
from pathlib import Path
import tifffile as tiff
import pandas as pd
from tqdm import tqdm

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## load the csv database

In [32]:
df = pd.read_csv(r'\\192.168.10.106\imdea\DataDriven_UT_AlbertoVicente\11_database\Database.csv')

df = df.replace({np.nan: None})

#get only the rows where 'Id' contains JI
df = df[df['Id'].str.contains('JI')]

df.head(5)

,Id,Original,Eq,Frontal 90 Right,XCT padded,Binary,Material Mask,Onlypores,UT,UT aligned,...,Registration Parameters,Registration Parameters lente,Dataset PatchvsVolfrac 3x3,Dataset PatchvsVolfrac lente 3x3,Dataset PatchvsVolfrac 5x5,Dataset PatchvsVolfrac lente 5x5,Dataset PatchvsVolfrac 7x7,Dataset PatchvsVolfrac lente 7x7,Dataset PatchvsVolfrac 9x9,Dataset PatchvsVolfrac lente 9x9
24,JI_4,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,None,None,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,...,[[ 9.99924865e-01 -1.22582733e-02 -7.75030259e...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None
25,JI_5,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,None,None,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,...,[[ 9.99942355e-01 -1.07371711e-02 -8.56068249e...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None
26,JI_7,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,None,None,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,...,[[ 9.99929001e-01 1.19160759e-02 -7.57121417e...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None
27,JI_8,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,None,None,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,...,[[ 9.99614862e-01 2.77511853e-02 -8.27453911e...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None,\\192.168.10.106\imdea\DataDriven_UT_AlbertoVi...,None
28,JI_9,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## Migrate

Now we just have to iterate the dataframe, retrieve the data and load it into the database

In [ ]:
#table name to load
table_name = 'datasets'

description = 'Monoelement RF vs volfrac and areafrac. First functional group of datasets from 2024'

targets = ['volfrac', 'areafrac']

patch_size = 9

#iterate over the rows of the dataframe

for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    #file loading

    file_path = row[f'Dataset PatchvsVolfrac {patch_size}x{patch_size}']

    if file_path is None:

        continue

    dataset = pd.read_csv(file_path)

    #file paths

    file_paths=[]

    file_paths.append(row['UT'])

    file_paths.append(row['Frontal 90 Right'])

    #metadata
    metadata_parameters = []

    #patch_size
    metadata_parameters.append({'key':'patch_size', 'value':patch_size, 'type':'pixels'})

    #target
    metadata_parameters.append({'key':'target', 'value':'volfrac', 'type':'nominal'})

    #target
    metadata_parameters.append({'key':'target2', 'value':'areafrac', 'type':'nominal'})

    #rows
    metadata_parameters.append({'key':'rows', 'value':len(dataset), 'type':'cardinal'})
    
    main_parameters = {'file_path':file_path}

    main_parameters['description'] = description

    try:

        #load the table

        # Extract column names and values from the attributes dictionary
        columns = ', '.join(main_parameters.keys())
        values = ', '.join([f"'{v}'" for v in main_parameters.values()])

        # Construct the SQL INSERT statement
        sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

        # Create a cursor object using the cursor() method
        cursor = conn.cursor()

        # Execute the SQL statement
        cursor.execute(sql)

        cursor.execute('COMMIT')

        cursor.close()

        #load the metadata

        data = qrs.get_data_metadata(table_name)

        row_id = data['id_dataset'].values[-1]

        metadata_table_name =  table_name[:-1] + '_metadata'

        for attributes in metadata_parameters:

            attributes[table_name[:-1] + '_id'] = row_id

            # Extract column names and values from the attributes dictionary
            columns = ', '.join(attributes.keys())
            values = ', '.join([f"'{v}'" for v in attributes.values()])

            # Construct the SQL INSERT statement
            sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

            # Create a cursor object using the cursor() method
            cursor = conn.cursor()

            # Execute the SQL statement
            cursor.execute(sql)

            cursor.execute('COMMIT')

            cursor.close()

        #load the measurement dataset table

        measurement_data = qrs.get_data_metadata('measurements')

        measurement_ids = []

        for path in file_paths:

            measurement_ids.append(measurement_data[measurement_data['file_path_measurement'] == str(path)]['id_measurement'].values[0])

        assert len(measurement_ids) == len(file_paths), 'The number of measurements is not the same as the number of file paths'

        relational_table_name = 'dataset_measurements'

        for measurement_id in measurement_ids:

            relational_parameters = {'dataset_id': row_id, 'measurement_id': measurement_id}

            # Extract column names and values from the attributes dictionary
            columns = ', '.join(relational_parameters.keys())
            values = ', '.join([f"'{v}'" for v in relational_parameters.values()])

            # Construct the SQL INSERT statement
            sql = f"INSERT INTO {relational_table_name} ({columns}) VALUES ({values})"

            # Create a cursor object using the cursor() method
            cursor = conn.cursor()

            # Execute the SQL statement
            cursor.execute(sql)

            cursor.execute('COMMIT')

            cursor.close()
            
    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error inserting row {index}: {error}")

100%|██████████| 8/8 [03:22<00:00, 25.32s/it]


In [34]:
conn.close()
print("Connection closed")

Connection closed
